# Del 4: Sampling og den diskrete Fouriertransformasjonen

I denne delen så kommer jeg til å introdusere den diskrete Fouriertransformasjonen. Fouriertransformasjonen som vi har hittil sett på er det som kalles den kontinuerlige Fouriertransformasjonen, som er definert for kontinuerlige funksjoner. Dette skaper da problemer når vi skal implementere Fouriertransformen på en datamaskin. Datamaskiner kan kun gjennomføre regneoperasjoner på diskrete verider, og derfor må vi ha en diskret versjon av Fouriertransformasjonen som gir en diskret resultat.

Den diskrete Fouriertransformasjonen, eller DFT er definert som følger:

$$X(k) = \sum_{n=0}^{N - 1} x(n)e^{-i\frac{2\pi nk}{N}}$$

Hvor $x(n)$ er en diskret funksjon, altså en tallrekke (ofte representert som en array i Python), og $X(k)$ er den diskrete Fouriertransformasjonen av $x(n)$. $N$ er antall punkter i $x(n)$, og $k$ er den k-te frekvensen i  $X(k)$. 

Prinsippet til DFT-en er da akkurat den samme som med den kontinuerlige Fouriertransformasjonen: vi tar en eller annen funksjon som befinner seg i tidsdomenet, og transformerer den til frekvensdomenet. Allikevel så er det noen nyanser med DFT-en som vi må være klar over. For at vi skal kunne forstå oss på disse nyansene, så må vi først se på hvordan man diskretiserer en kontinuerlig funksjon, også kalt sampling:

## Sampling



Sampling er når vi tar en kontinuerlig funksjon og gjør den diskret. Intuitivt så kan vi si at vi deler opp den kontinuerlige funksjonen i biter med lik intervall mellom seg, og så tar vi verdien til funksjonen i disse punktene: 

<img src="plots/p4_cont.png" width="400px"> <img src="plots/p4_cont_samp.png" width="400px"> <img src="plots/p4_samp.png" width="400px"> 

Etter vi har gjort dette har vi en diskret tallrekke som kan lagres på og bli behandlet av en datamaskin. 

La oss introdusere noen nye begreper: 

- $N$: Total antall samplepunkter i den diskrete funksjonen. 
- Samplingsintervall: Også kjent som samplingsperiode. Dette er avstanden mellom hvert samplepunkt, og er ofte betegnet med $T_s$, med sekunder som enhet.
- Samplingsfrekvens: Dette er antall samplepunkter per sekund, og er ofte betegnet med $f_s$, med enheten Hz. Samplingsfrekvensen er da den inverse av samplingsintervallet: $f_s = \frac{1}{T_s}$

Hvis vi da tar for oss en sinusfunksjon som vi pleier å se i Python:
```python
t = np.linspace(0, 10, 1000) # tid fra 0 til 2 sekunder med 1000 samplepunkter
sinus = np.sin(t)
```

er $N = 1000$, samplingsintervallet er $T_s = 10/1000 = 0.01$ sekunder, og samplingsfrekvensen er $f_s = 1/T_s = 100$ Hz.

Konverteringen fra kontinuerlig til diskret er en prosess som kalles for anolog til digital konvertering, og er noe som skjer på hardwarenivå, for eksempel når du har en mikrofon som tar inn en lydbølge (en anolog / kontinuerlig verdi) og konverterer den til en digital tallrekke ved å sample den. Vi gjør noe lignende i kodesnutten overnfor, hvor vi definerer en diskret tallrekke med N antall samplingspunkter med linspace-funksjonen, og bruker det som input til sinusfunksjonen. Da får vi en diskret sinusfunksjon.


## Undersampling og Nyquists samplingsteorem

Hva skjer hvis vi har en kontinuerlig funksjon, og vi prøver å sample den med en samplingsfrekvens som er for lav? La oss ta sinusfunksjonen fra kodesnutten ovenfor som en eksempel:

```python
t = np.linspace(0, 10, 1000) 
sinus = np.sin(t)
```

Hvis vi plotter den rett ut så ser den slik ut:

<img src="plots/p4_sine.png" width="400px">

Ser ut som en helt vanlig sinusfunksjon. La oss nå minke samplingsfrekvensen, altså øke samplingsintervallet til noe banalt: 

```python
t = np.linspace(0, 10, 3) 
sinus = np.sin(t)
```

Vi har altså en sinusfunksjon med en frekvens på $2\pi$ Hz, som vi skal prøve å sample med kun tre punkter fordelt på 10 sekunder. Samplingsintervallet er nå $T_s = 10/3$ sekunder, og samplingsfrekvensen er $f_s = 1/T_s = 3/10$ Hz. Hvis vi plotter denne funksjonen så får vi:

<img src="plots/p4_sine_usamp.png" width="400px">

Dette ser jo ikke ut som en sinusfunksjon i det hele tatt! Her kan vi bemerke at plottet er særdeles stygg for å være en sinusfunksjon, og at vi mister informasjon om hvor ofte funksjonen svinger. I det "fine" plottet så ser vi tydelig at funksjonen svinger ca. 1.5 ganger i løpet av 10 sekunder, mens ut ifra det stygge plottet så kan vi fort tro at den svinger mindre enn en gang i løpet av samme tid.

Dette kalles for undersampling. Vi har altså en funksjon med en eller annen frekvenskomponent, i dette tilfellet kun en frekvenskomponent med frekvens lik $2\pi$ Hz, og vi sampler den med en samplingsfrekvens som er for lav. Dette fører til at vi mister informasjon om frekvenskomponentene til funksjonen, og vi får en helt annen funksjon enn den vi hadde i utgangspunktet. 

Men hva er "for lav" samplingsfrekvens? Altså, hva er den laveste samplingsfrekvensen vi kan ha før vi begynner å miste informasjon om funksjonen? Da er vi inne på **Nyquists samplingsteorem**, som sier at for å unngå undersampling, så **må samplingsfrekvensen være minst dobbelt så stor som den høyeste frekvenskomponenten i funksjonen**. Den høyeste frekvensen som man kan sample uten å miste informasjon kalles for Nyquist-frekvensen, som er da halvparten av samplingsfrekvensen.

Hvis vi tar for oss en funksjon som er en sum av to sinusfunksjoner med frekvenser 1 Hz og 2 Hz, så må samplingsfrekvensen være minst 4 Hz for å unngå undersampling. Med en samplingsfrekvens på 4 Hz så vil vi få en Nyquist-frekvens på 2 Hz.

## Konsekvenser av sampling i frekvensdomenet 

Så langt så har vi sett på sampling og undersampling i tidsdomenet. Men har undersampling, og det at vi i det hele tatt har en diskret funksjon, noen konsekvenser i frekvensdomenet? Svaret er ja, og det er akkurat dette som skiller resultatet til DFT-en fra den kontinuerlige Fouriertransformasjonen. 


### Periodisering

